In [81]:
!pip install lightgbm numpy pandas rdkit scikit-learn #在终端用pip(不用写感叹号)下载lightgbm库、numpy库、pandas库、rdkit库、scikit-learn库

In [82]:
import lightgbm as lgb  # 导入机器学习模型LightGBM
import numpy as np  # 导入数值计算库NumPy
import pandas as pd  # 导入数据处理库Pandas

from rdkit import Chem  # 导入RDKit中的Chem模块，用于分子对象转换
from rdkit.Chem.rdMolDescriptors import GetMorganFingerprintAsBitVect  # 从RDKit中导入GetMorganFingerprintAsBitVect函数，用于生成分子指纹（位向量转换）

from sklearn.model_selection import train_test_split  # 从scikit-learn中导入train_test_split函数，用于拆分数据集
from sklearn.metrics import fbeta_score  # 从scikit-learn中导入fbeta_score函数，用于 F2 Score 计算

In [83]:
DATA_PATH = "data/"  # 数据路径

raw_data = pd.read_csv(f"{DATA_PATH}/mol_train.csv")  # 读入训练数据
test_data = pd.read_csv(f"{DATA_PATH}/mol_test.csv")  # 读入测试数据

# 拆分训练数据为训练集与验证集，验证集占比 20%，设定固定随机种子
train_data, valid_data = train_test_split(
    raw_data, test_size=0.2, random_state=hash("Datawhale") % 2023
)

In [84]:
def smile2fingerprint(smile: str):
    """将 SMILE 分子式表示为指纹数据

    参数:
        smile (string): SMILE 分子式

    返回:
        fp (Explict BitVect): 分子式的 Morgan 指纹位向量
    """
    molecular = Chem.MolFromSmiles(smile)  # 将字符串转换为分子式对象
    finger_print = GetMorganFingerprintAsBitVect(molecular, 2, nBits=1024)  # 获得分子式的 Morgan 指纹位向量
    return finger_print

In [85]:
# 批量将位向量转换为特征矩阵，形状为 (n, 1024)，n 代表数据个数，1024 在位向量转换时指定
train_X = np.array([smile2fingerprint(smile) for smile in train_data["SMILES"]])
valid_X = np.array([smile2fingerprint(smile) for smile in valid_data["SMILES"]])
test_X = np.array([smile2fingerprint(smile) for smile in test_data["SMILES"]])

In [86]:
# 将数据特征矩阵转换为 LightGBM 指定格式，(特征向量，对应标签)
lgb_train = lgb.Dataset(train_X, label=train_data["TARGET"])
lgb_valid = lgb.Dataset(valid_X, label=valid_data["TARGET"])

**模型训练**

In [87]:
# 设定 LightGBM 训练参，查阅参数意义：https://lightgbm.readthedocs.io/en/latest/Parameters.html
lgb_params = {
    "objective": "binary",  # 指定任务类别为二分类
    "seed": hash("Datawhale") % 2023,  # 设定随机种子
    "verbose": -1,  # 禁用输出（可选）
}

# 训练模型，参数依次为：导入模型设定参数、导入训练集、设定模型迭代次数（100）、导入验证集
model = lgb.train(lgb_params, lgb_train, num_boost_round=300, valid_sets=lgb_valid)

In [88]:
threshold = 0.4 # 模型输出的是类别概率，设定概率的判断阙值

In [89]:
# 用验证集进行模型预测（选择训练中最好的一次）
valid_pred = model.predict(valid_X, num_iteration=model.best_iteration)
# 生成预测标签结果，如果概率大于阈值则为 1，否则为 0
valid_result = [1 if x > threshold else 0 for x in valid_pred]
# 计算验证集 F2 Score 分数
valid_score = fbeta_score(valid_data["TARGET"], valid_result, beta=2)
print(f"Valid Score: {valid_score}")

Valid Score: 0.745967741935484


In [90]:
# 预测测试集数据并获得预测结果
pred = model.predict(test_X, num_iteration=model.best_iteration)
result = [1 if x > threshold else 0 for x in pred]

In [91]:
submission = pd.DataFrame()  # 创建预测结果 DataFrame
submission["SMILES"] = test_data["SMILES"]  # 复制对应的 SMILES 内容
submission["TARGET"] = result  # 填写预测标签结果
submission.to_csv("./submission.csv", index=False)  # 保存最后的预测结果到 submission.csv